In [1]:
import os
from ciao_contrib.runtool import *
import glob
import pycrates as pcr
from paramio import *
from crates_contrib.utils import *
import numpy as np

objList = sorted(glob.glob("NGC*"))
#print(objList)

for obj in objList[20:21:]:
    fileList = glob.glob("{}/*/repro/*reproj*".format(obj))
    print(fileList)
    cr = pcr.read_file('{}/sources/merged_0.3-7_offaxs_src.fits'.format(obj))
    obscount = len(fileList)
    #print(obscount)
    obsids = []
    for file in fileList:
        obsList = file.split('/')
        obsids.append(obsList[1])
    print(obsids)
    exptimes = []
    offaxscol = [[] for i in range(obscount)]
    for j in range(obscount):
        #finds exposure for x number of observations
        exptimes.append(dmkeypar(fileList[j], keyword = 'LIVETIME', echo = 'yes'))
        #gets the off-axis angles from file
        offaxscol[j] = pcr.get_colvals(cr,'{}_OFFAXS_ANG'.format(obsids[j]))
    print(exptimes)
    #print(offaxscol[0])
    weight = [[] for i in range(obscount)]
    floexptime = []
    for k in range(obscount):
        #print(offaxscol[k])
        #print(exptimes[k])
        #calculates sum of exposure times by off-axis angles
        weight[k] = np.array(offaxscol[k])*float(exptimes[k])
        floexptime.append(float(exptimes[k]))
        #print(avgsum)
        #divides sum of exposure times*off-axis by angles sum of exposure times
    #print(sum(intexptime))
    weightavg = np.array(sum(weight))/(sum(floexptime))
    #print(weightavg)
    raerrcol = pcr.get_colvals(cr,'RA_ERR')*60
    #print(raerrcol)
    decerrcol = pcr.get_colvals(cr,'DEC_ERR')*60
    #print(decerrcol)
    newerr = np.sqrt(raerrcol**2 + decerrcol**2 + ((0.2/60) + ((1.4/60) * (weightavg/8)**2))**2)
    #print(newerr)
    wangerr = ((0.2/60) + ((1.4/60) * (weightavg/8)**2))
    cr = make_table_crate(weightavg,wangerr,newerr,colnames=['WEIGHT_OFFAXS','WANG_ERR','POS_ERR'])
    poserrcol = cr.get_column('POS_ERR')
    weightoffaxs = cr.get_column('WEIGHT_OFFAXS')
    wangcol = cr.get_column('WANG_ERR')
    poserrcol.unit = 'arcmin'
    weightoffaxs.unit = 'arcmin'
    wangcol.unit = 'arcmin'
    pcr.write_file(cr,"{}/sources/newerr.fits".format(obj),clobber=True)
    dmpaste (infile="{}/sources/merged_0.3-7_offaxs_src.fits[SRCLIST]".format(obj),pastefile="{}/sources/newerr.fits".format(obj),outfile="{}/sources/merged_0.3-7_newerr_src.fits".format(obj),clobber=True)

['NGC4594/1586/repro/1586_reproj_evt.fits', 'NGC4594/407/repro/407_reproj_evt.fits', 'NGC4594/9532/repro/9532_reproj_evt.fits', 'NGC4594/9533/repro/9533_reproj_evt.fits']
['1586', '407', '9532', '9533']
[18518.17509487, 1766.3764380596, 84914.523107418, 88968.896862404]


In [174]:
print((2.885764191027129*27698.409300435)+(2.432194250837314*5068.7356162665)+(2.796192931641599*1916.7743489704)+(2.640862066648127*9994.4323488192)+(2.886654130838732*22763.310169151))

189722.6188176694


In [176]:
print(189722.6188176694/(27698.409300435+5068.7356162665+1916.7743489704+9994.4323488192+22763.310169151))

2.813136773324384


In [184]:
print((0.2/60 + (1.4/60 * (2.813136773324384/8)**2)))

0.006218550496771328


In [178]:
print(3.683522061947997e-05*60,4.17338640357201e-05*60)

0.002210113237168798 0.002504031842143206


In [179]:
print(np.sqrt(((3.683522061947997e-05*60)**2) + ((4.17338640357201e-05*60)**2)))

0.0033398766425686804


In [182]:
print(np.sqrt((3.683522061947997e-05*60)**2 + (4.17338640357201e-05*60)**2 + (0.2/60 + (1.4/60 * (2.813136773324384/8)**2)**2)))

0.05790347647077079
